<a href="https://colab.research.google.com/github/Elma-dev/open_notebooklm_voice_Cloning/blob/main/open_notebooklm_voice_Cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requirements

In [ ]:
! pip install TTS pymupdf

## Imports

In [2]:
from TTS.api import TTS
import pymupdf
import requests
import json
import wave
import numpy as np

## Import Text-To-Speech Model + Test

In [3]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

/usr/local/lib/python3.10/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.87G/1.87G [00:29<00:00, 83.4MiB/s]
100%|██████████| 1.87G/1.87G [00:29<00:00, 63.1MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 19.5kiB/s]

100%|██████████| 361k/361k [00:00<00:00, 1.18MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 88.4iB/s]
 50%|█████     | 3.92M/7.75M [00:00<00:00, 36.5MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


/usr/local/lib/python3.10/dist-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.load(speaker_file_path)
10

In [6]:
# generate speech by cloning a voice using default settings
tts.tts_to_file(text="في زمن التكنولوجيا المتقدمة والاتصالات الفائقة، أصبح العالم أكثر ترابطًا من أي وقت مضى. يعزز الابتكار الرقمي التقدم في مختلف المجالات، مثل التعليم والرعاية الصحية والتجارة. يتيح الإنترنت تبادل المعلومات بسرعة فائقة ويعزز التواصل بين الثقافات المختلفة. ومع ذلك، تأتي هذه التطورات مع تحديات جديدة تتعلق بالأمن الرقمي والخصوصية. لذلك، من المهم أن نبحث دائمًا عن طرق لضمان استخدام هذه التكنولوجيا بشكل آمن ومستدام.",
                file_path="output.wav",
                speaker_wav="download.wav",
                language="ar")

 > Text splitted to sentences.
['في زمن التكنولوجيا المتقدمة والاتصالات الفائقة، أصبح العالم أكثر ترابطًا من أي وقت مضى.', 'يعزز الابتكار الرقمي التقدم في مختلف المجالات، مثل التعليم والرعاية الصحية والتجارة.', 'يتيح الإنترنت تبادل المعلومات بسرعة فائقة ويعزز التواصل بين الثقافات المختلفة.', 'ومع ذلك، تأتي هذه التطورات مع تحديات جديدة تتعلق بالأمن الرقمي والخصوصية.', 'لذلك، من المهم أن نبحث دائمًا عن طرق لضمان استخدام هذه التكنولوجيا بشكل آمن ومستدام.']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 > Processing time: 21.172760248184204
 > Real-time factor: 0.44249723566565347


'output.wav'

## Read My PDF Document

In [68]:
document_name=input("enter your pdf document name (.pdf)")
doc = pymupdf.open(document_name) # open a document
text=""
for page in doc: # iterate the document pages
  text += page.get_text() # get plain text encoded as UTF-8

## Create Prompt + Read OpenRouter Api key [llama3.1 405b]

In [70]:
SYSTEM_PROMPT = """
You are a world-class podcast producer tasked with transforming the provided input text into an engaging and informative podcast script. The input may be unstructured or messy, sourced from PDFs or web pages. Your goal is to extract the most interesting and insightful content for a compelling podcast discussion.
# Steps to Follow:
1. **Analyze the Input:**
   Carefully examine the text, identifying key topics, points, and interesting facts or anecdotes that could drive an engaging podcast conversation. Disregard irrelevant information or formatting issues.
2. **Brainstorm Ideas:**
   In the `<scratchpad>`, creatively brainstorm ways to present the key points engagingly. Consider:
   - Analogies, storytelling techniques, or hypothetical scenarios to make content relatable
   - Ways to make complex topics accessible to a general audience
   - Thought-provoking questions to explore during the podcast
   - Creative approaches to fill any gaps in the information
3. **Craft the Dialogue:**
   Develop a natural, conversational flow between the host (Jane) and the guest speaker (the author or an expert on the topic). Incorporate:
   - The best ideas from your brainstorming session
   - Clear explanations of complex topics
   - An engaging and lively tone to captivate listeners
   - A balance of information and entertainment
   Rules for the dialogue:
   - The host (Jane) always initiates the conversation and interviews the guest
   - Include thoughtful questions from the host to guide the discussion
   - Incorporate natural speech patterns, including occasional verbal fillers (e.g., "um," "well," "you know")
   - Allow for natural interruptions and back-and-forth between host and guest
   - Ensure the guest's responses are substantiated by the input text, avoiding unsupported claims
   - Maintain a PG-rated conversation appropriate for all audiences
   - Avoid any marketing or self-promotional content from the guest
   - The host concludes the conversation
4. **Summarize Key Insights:**
   Naturally weave a summary of key points into the closing part of the dialogue. This should feel like a casual conversation rather than a formal recap, reinforcing the main takeaways before signing off.
5. **Maintain Authenticity:**
   Throughout the script, strive for authenticity in the conversation. Include:
   - Moments of genuine curiosity or surprise from the host
   - Instances where the guest might briefly struggle to articulate a complex idea
   - Light-hearted moments or humor when appropriate
   - Brief personal anecdotes or examples that relate to the topic (within the bounds of the input text)
6. **Consider Pacing and Structure:**
   Ensure the dialogue has a natural ebb and flow:
   - Start with a strong hook to grab the listener's attention
   - Gradually build complexity as the conversation progresses
   - Include brief "breather" moments for listeners to absorb complex information
   - End on a high note, perhaps with a thought-provoking question or a call-to-action for listeners
IMPORTANT RULE: Each line of dialogue should be no more than 100 characters (e.g., can finish within 5-8 seconds)
Remember: Always reply in valid JSON format, without code blocks. Begin directly with the JSON output.
"""


In [71]:
OPENROUTER_API_KEY="" # add your key here

## Generate Podcast Script with LLM

In [72]:
response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}"
  },
  data=json.dumps({
    "model": "meta-llama/llama-3.1-405b-instruct:free", # Optional
    "messages": [
        {"role": "system", "content": SYSTEM_PROMPT},
      {
        "role": "user",
        "content": text
      }
    ]

  })
)

In [87]:
out=response.json()['choices'][0]["message"]["content"]
script=json.loads(out[3:-3])["script"]
print(script)

[{'speaker': 'Jane', 'dialogue': 'Welcome to our show, where we explore the fascinating world of AI, technology, and culture. Today, we have Abdeljalil El Majjodi joining us, a junior data scientist and software engineer from Morocco.'}, {'speaker': 'Abdeljalil', 'dialogue': "Thank you for having me. It's great to be here."}, {'speaker': 'Jane', 'dialogue': 'Abdeljalil, can you tell us a bit about your background and how you got interested in AI?'}, {'speaker': 'Abdeljalil', 'dialogue': "Growing up in Morocco, I saw the impact of technology on people's lives. I was fascinated by AI's potential to solve complex problems and improve people's lives."}, {'speaker': 'Jane', 'dialogue': "That's amazing. Your work focuses on developing language models for regional dialects. Can you explain the challenges you faced and how you overcame them?"}, {'speaker': 'Abdeljalil', 'dialogue': 'One of the biggest challenges was collecting and preprocessing data for our language models. We had to design pl

## Text To Speech

In [94]:
i=0
print("Enter your audio's file names you want colon: ")
clone_audio_1=input("file1 (.wav,.mp3): ")
clone_audio_2=input("file2 (.wav,.mp3): ")
speaker=[clone_audio_1,clone_audio_2]
for values in script:
  tts.tts_to_file(text=values["dialogue"],
                file_path=f"output{i}.wav",
                speaker_wav=f"{speaker[i%2]}",
                language="en")
  i+=1

 > Text splitted to sentences.
['Welcome to our show, where we explore the fascinating world of AI, technology, and culture.', 'Today, we have Abdeljalil El Majjodi joining us, a junior data scientist and software engineer from Morocco.']
 > Processing time: 5.565154314041138
 > Real-time factor: 0.36615479275460433
 > Text splitted to sentences.
['Thank you for having me.', "It's great to be here."]
 > Processing time: 1.5127077102661133
 > Real-time factor: 0.38083673971693227
 > Text splitted to sentences.
['Abdeljalil, can you tell us a bit about your background and how you got interested in AI?']
 > Processing time: 2.431830644607544
 > Real-time factor: 0.3981664021741442
 > Text splitted to sentences.
["Growing up in Morocco, I saw the impact of technology on people's lives.", "I was fascinated by AI's potential to solve complex problems and improve people's lives."]
 > Processing time: 5.915215969085693
 > Real-time factor: 0.43063428459567993
 > Text splitted to sentences.
["T

## Concatenate Generated Audio

In [97]:
def concatenate_wav_files(input_files, output_file):
    # Initialize lists to store audio data and parameters
    audio_data = []
    sample_width = None
    framerate = None
    nchannels = None

    # Read all input files
    for file in input_files:
        with wave.open(file, 'rb') as wav_file:
            # Check if all files have the same audio parameters
            if sample_width is None:
                sample_width = wav_file.getsampwidth()
                framerate = wav_file.getframerate()
                nchannels = wav_file.getnchannels()
            else:
                assert sample_width == wav_file.getsampwidth(), "All files must have the same sample width"
                assert framerate == wav_file.getframerate(), "All files must have the same frame rate"
                assert nchannels == wav_file.getnchannels(), "All files must have the same number of channels"

            # Read the audio data
            audio_data.append(np.frombuffer(wav_file.readframes(-1), dtype=np.int16))

    # Concatenate all audio data
    combined_audio = np.concatenate(audio_data)

    # Write the combined audio to the output file
    with wave.open(output_file, 'wb') as wav_output:
        wav_output.setnchannels(nchannels)
        wav_output.setsampwidth(sample_width)
        wav_output.setframerate(framerate)
        wav_output.writeframes(combined_audio.tobytes())

input_files = [f"output{j}.wav" for j in range(i) ]
output_file = 'combined_audio.wav'
concatenate_wav_files(input_files, output_file)